In [1]:
import ee
import geemap 

from ykwmp.rsd import Sentinel2

ee.Initialize()

In [3]:
aoi = ee.FeatureCollection("projects/yk-wetland-mapping/assets/yukon-bounds")

s2_2018 = Sentinel2(aoi, '2018-06-20', '2018-09-21').process()
s2_2019 = Sentinel2(aoi, '2019-06-20', '2019-09-21').process()
s2_2020 = Sentinel2(aoi, '2020-06-20', '2020-09-21').process()

s2 = s2_2018.merge(s2_2019).merge(s2_2020)

s2_composite = s2.composite()

In [4]:
Map = geemap.Map()
vis = {'bands': ['B4', 'B3', 'B2'],'min': 0, 'max': 3000}
Map.addLayer(s2_composite, vis, f'S2 (B4, B3, B2): 2018-2020')
Map.addLayerControl()

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…